In [8]:
import tensorflow as tf
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

from dataGenerator import make_generator

In [16]:
def station_conv(mag_input):
    """extracts features from the magnetic field information
    """
    mag_features = mag_input
    return mag_features


In [10]:
def station_net(mag_features, st_loc):
    """aggregates information from magnetic field data and positions
    """
    st_features = mag_features
    return st_features


In [11]:
def global_conv(global_params):
    """extracts features from the global parameters (solar wind, etc.)
    """
    global_features = None
    return global_features


In [12]:
def aggregator(st_features, global_features):
    """adds up the station vectors, concatenates with the global features,
    outputs the substorm prediction
    """
    model_output = st_features
    return model_output


In [13]:
def loss_function(model_output, ss_occurred, ss_time, ss_loc):
    """calculates loss based on model output and the substorm occurance
    """
    loss = 0
    return loss

In [17]:
os.chdir("C://Users//Greg//code//space physics//experiment//data")

mag_files = glob.glob("mag_data*.nc")
ss_file = ss_file = "substorms_2000_2018.csv"
stats_file = "statistics.npz"
data_interval = 120
prediction_interval = 250

gen = make_generator(mag_files, ss_file, stats_file, data_interval, prediction_interval)

dataset = tf.data.Dataset.from_generator(gen,
                                         output_types = (tf.float32, tf.float32, tf.bool, tf.float32, tf.float32),
                                         output_shapes = (tf.TensorShape([None, data_interval+1, 3]),
                                                          tf.TensorShape([None, 3]),
                                                          tf.TensorShape([]),
                                                          tf.TensorShape([]),
                                                          tf.TensorShape([3]))
                                        )

dataset = dataset.repeat()

data_iter = dataset.make_initializable_iterator()
mag_input, st_loc, ss_occurred, ss_time, ss_loc = data_iter.get_next()
global_params = None

mag_features = station_conv(mag_input)
st_features = station_net(mag_features, st_loc)
global_features = global_conv(global_params)
model_output = aggregator(st_features, global_features)

loss = loss_function(model_output, ss_occurred, ss_time, ss_loc)

train_step = tf.train.AdamOptimizer().minimize(loss)

STEPS = 10

sess = tf.Session()
with tf.device("/gpu:0"):
    sess.run(tf.global_variables_initializer())
    sess.run(data_iter.initializer)
    for _ in range(STEPS):
        _, loss_val = sess.run([train_step, loss])
        print(loss_val)

AttributeError: 'int' object has no attribute 'dtype'